# Set up the notebook by importing and defining functions

In [1]:
% pylab notebook
from datacube.storage import masking
from datetime import datetime
from skimage import exposure
import datacube

import sys
import os
sys.path.append(os.path.abspath('/g/data/r78/cek156/dea-notebooks/algorithms'))
import DEAPlotting
import DEADataHandling
import BandIndices

dc = datacube.Datacube(app='crops')

Populating the interactive namespace from numpy and matplotlib


# Load some data

In [2]:
query = {
        'lat': (-35.51, -35.69),
        'lon': (146.81, 147.19),
        'time':('2017-01-01', '2017-12-31')
        }

test_area = dc.load(product='ls8_nbar_albers', group_by='solar_day', **query)

In [3]:
test_area

<xarray.Dataset>
Dimensions:          (time: 44, x: 1461, y: 957)
Coordinates:
  * time             (time) datetime64[ns] 2017-01-06T00:02:51 ...
  * y                (y) float64 -3.956e+06 -3.957e+06 -3.957e+06 -3.957e+06 ...
  * x                (x) float64 1.336e+06 1.336e+06 1.336e+06 1.336e+06 ...
Data variables:
    coastal_aerosol  (time, y, x) int16 598 604 606 581 588 582 579 574 569 ...
    blue             (time, y, x) int16 676 685 689 647 659 664 646 633 623 ...
    green            (time, y, x) int16 993 1002 1016 957 963 959 936 918 ...
    red              (time, y, x) int16 1321 1344 1361 1296 1295 1285 1253 ...
    nir              (time, y, x) int16 2160 2138 2132 2092 2072 2028 2025 ...
    swir1            (time, y, x) int16 3523 3502 3484 3531 3524 3471 3491 ...
    swir2            (time, y, x) int16 2503 2486 2456 2475 2476 2479 2470 ...
Attributes:
    crs:      EPSG:3577

In [5]:
LandsatNDWI = BandIndices.calculate_indices(test_area, 'NDPI')

Hmmmmm. I don't recognise that index. Options I currently have are NDVI and NDWI.


<xarray.DataArray (time: 44, y: 957, x: 1461)>
array([[[-0.370124, -0.361783, ..., -0.462133, -0.485655],
        [-0.35873 , -0.348763, ..., -0.476589, -0.379449],
        ..., 
        [-0.478309, -0.457061, ..., -0.512012, -0.535326],
        [-0.471666, -0.474082, ..., -0.459272, -0.526514]],

       [[-0.      , -0.      , ..., -0.455983, -0.501684],
        [-0.      , -0.      , ..., -0.460575, -0.4282  ],
        ..., 
        [-0.      , -0.      , ..., -0.487242, -0.553918],
        [-0.      , -0.      , ..., -0.442853, -0.525689]],

       ..., 
       [[-0.      , -0.      , ..., -0.470178, -0.443866],
        [-0.      , -0.      , ..., -0.403881, -0.280521],
        ..., 
        [-0.      , -0.      , ..., -0.510519, -0.525687],
        [-0.      , -0.      , ..., -0.448985, -0.488463]],

       [[-0.019541, -0.020289, ..., -0.004937, -0.005477],
        [-0.019916, -0.01977 , ..., -0.004649, -0.005362],
        ..., 
        [-0.039167, -0.037685, ..., -0.015479, -0.01

In [ ]:
DEAP(test_area, bands = ['red', 'green', 'blue'], num_cols = 3, figsize = [10, 30])

# Grab the accompanying pixel quality data

In [ ]:
test_area_pq = dc.load(product='ls8_pq_albers', group_by='solar_day', fuse_func = ga_pq_fuser, **query)

In [ ]:
test_area_pq

In [ ]:
mask_components = {'cloud_acca':'no_cloud',
'cloud_shadow_acca' :'no_cloud_shadow',
'cloud_shadow_fmask' : 'no_cloud_shadow',
'cloud_fmask' :'no_cloud',
'blue_saturated' : False,
'green_saturated' : False,
'red_saturated' : False,
'nir_saturated' : False,
'swir1_saturated' : False,
'swir2_saturated' : False,
'contiguous':True}

In [ ]:
quality_mask = masking.make_mask(test_area_pq, **mask_components)

In [ ]:
quality_mask

## Check the mask looks ok

In [ ]:
quality_mask.isel(time = 3).pixelquality.plot(figsize = [10, 10])

## Apply the mask to the Landsat data

In [ ]:
cleaned_data = test_area.where(quality_mask.pixelquality)

In [ ]:
cleaned_data

# Load in WOFS for the same location

In [ ]:
dcwofs = datacube.Datacube(config='/home/156/cek156/wofscube.conf')

query = {
        'lat': (-35.51, -33.66),
        'lon': (146.81, 151.43)
        }

test_area_wofs = dcwofs.load(product='old_wofs', **query)

In [ ]:
test_area_wofs

In [ ]:
dcwofs.list_products()